In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from matplotlib import pyplot as plt

import medmnist
from medmnist import INFO, Evaluator

In [2]:
lap_egns_1 = torch.load("eigenvalues/lap_egns_1_up_pneum.pt")
lap_egns_2 = torch.load("eigenvalues/lap_egns_2_up_pneum.pt")
lap_egns_3 = torch.load("eigenvalues/lap_egns_3_up_pneum.pt")
lap_egns_4 = torch.load("eigenvalues/lap_egns_4_up_pneum.pt")
lap_egns_5 = torch.load("eigenvalues/lap_egns_5_up_pneum.pt")

lap_egns_test = torch.load("eigenvalues/lap_egns_test_up_pneum.pt")

train_labels = torch.load("eigenvalues/train_labels_pneum.pt")
test_labels = torch.load("eigenvalues/test_labels_pneum.pt")

In [3]:
train_eigns = torch.cat([lap_egns_1, lap_egns_2, lap_egns_3, lap_egns_4, lap_egns_5])

In [4]:
train_eigns.shape

torch.Size([4708, 14])

In [5]:
train_data = [(lap, lb) for lap, lb in zip(train_eigns, train_labels.squeeze().tolist())]  # train_eigns[:, :, 1] 

In [6]:
test_data = [(lap, lb) for lap, lb in zip(lap_egns_test, test_labels.squeeze().tolist())] # lap_egns_test[:, :, 1] 

# Training

In [7]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
                                           shuffle=True)

In [8]:
train_eigns.shape

torch.Size([4708, 14])

In [9]:
model = nn.Sequential(
            nn.Linear(14, 2048),
            nn.Tanh(),
            nn.Linear(2048, 2048),
            nn.Tanh(),
            nn.Linear(2048, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 256),
            nn.Tanh(),
            nn.Linear(256, 2),
            # nn.LogSoftmax(dim=1)
)

In [10]:
learning_rate = 5e-3

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 150

In [11]:
for epoch in range(n_epochs):
    for eig_vals, label in train_loader:
        out = model(eig_vals)
        loss = loss_fn(out, label)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    # if loss <= 0.1:
    #     break

Epoch: 0, Loss: 0.672526
Epoch: 1, Loss: 0.516529
Epoch: 2, Loss: 0.641875
Epoch: 3, Loss: 0.588324
Epoch: 4, Loss: 0.535340
Epoch: 5, Loss: 0.580469
Epoch: 6, Loss: 0.493762
Epoch: 7, Loss: 0.507230
Epoch: 8, Loss: 0.559044
Epoch: 9, Loss: 0.470834
Epoch: 10, Loss: 0.582531
Epoch: 11, Loss: 0.560940
Epoch: 12, Loss: 0.508982
Epoch: 13, Loss: 0.571094
Epoch: 14, Loss: 0.564142
Epoch: 15, Loss: 0.556890
Epoch: 16, Loss: 0.513338
Epoch: 17, Loss: 0.719438
Epoch: 18, Loss: 0.547533
Epoch: 19, Loss: 0.497279
Epoch: 20, Loss: 0.515987
Epoch: 21, Loss: 0.536389
Epoch: 22, Loss: 0.546016
Epoch: 23, Loss: 0.504062
Epoch: 24, Loss: 0.618706
Epoch: 25, Loss: 0.621936
Epoch: 26, Loss: 0.531741
Epoch: 27, Loss: 0.430661
Epoch: 28, Loss: 0.572491
Epoch: 29, Loss: 0.418419
Epoch: 30, Loss: 0.462196
Epoch: 31, Loss: 0.366145
Epoch: 32, Loss: 0.492072
Epoch: 33, Loss: 0.399918
Epoch: 34, Loss: 0.474991
Epoch: 35, Loss: 0.590050
Epoch: 36, Loss: 0.578634
Epoch: 37, Loss: 0.428061
Epoch: 38, Loss: 0.416

In [12]:
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
#                                            shuffle=False)
correct = 0
total = 0

with torch.no_grad():
    for laps, labels in train_loader:
        outputs = model(laps)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.905905


In [13]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for laps, labels in test_loader:
        outputs = model(laps)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.858974
